In [3]:
import pandas as pd
import scanpy as sc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from sklearn.metrics.pairwise import cosine_similarity
from itertools import product
from pathlib import Path

In [8]:
# Load peak–gene distance file (BED-like format, tab-delimited, no header)
distances=pd.read_csv("./distance_bed.txt", delimiter="\t", header=None)

# Construct peak name in standard format: chr:start-end
distances["peak_name"]=distances[0] + ":" + distances[1].astype(str) + "-" + distances[2].astype(str)

# Compute minimum absolute distance to gene (columns 4 and 5 contain signed distances)
distances['min_distance'] = np.abs(distances[[4, 5]]).min(axis=1)

# Keep only relevant columns
distances=distances.rename({ 3:"genes"}, axis=1)
distances_orig=distances[["genes", "peak_name", "min_distance"]]


# Preview
distances_orig.head()

,genes,peak_name,min_distance
0,Xkr4,chr1:3119557-3120226,551272
1,Xkr4,chr1:3120334-3120722,550776
2,Xkr4,chr1:3292408-3293249,378249
3,Xkr4,chr1:3309766-3310590,360908
4,Xkr4,chr1:3311241-3312081,359417


In [12]:
# Hyperparameter settings
              # Mini-batch sizes used during training
               # Number of latent programs (K)
LAMBDA1_VALUES = [1, 3, 5, 7, 9, 11, 13, 15]  # Regularization strengths for lambda1                 # Simulation replicate IDs

BASE_DIR = Path("./new_sim")         # Base directory containing simulation outputs
N_PROGRAMS = 32                      # Default maximum number of programs

# Store contrast score results for all parameter combinations
results = []

# Iterate over all combinations of replicate, K, lambda1, and batch size
for lambda1 in product(LAMBDA1_VALUES):

    # Construct directory path for the current parameter combination
    run_dir = (
        BASE_DIR
        / f"output_sim_p_rep_45"
        / f"lambda1_{lambda1}_bs_1024_k_32"
    )

    # Paths to required output files
    atac_path = run_dir / "spatial-ATAC.h5ad"
    weights_path = run_dir / "weights.pt"

    # Skip if required files are missing
    if not atac_path.exists() or not weights_path.exists():
        continue

    # Load trained model weights (CPU mode for portability)
    checkpoint = torch.load(weights_path, map_location="cpu")

    # Use K-specific number of programs
    N_PROGRAMS = 32

    # Extract peak (W1) and gene (W2) program loadings
    W2 = checkpoint["W2"].numpy()[:, :N_PROGRAMS]
    W1 = checkpoint["W1"].numpy()[:, :N_PROGRAMS]

    # Load corresponding spatial RNA data
    rna_path = run_dir / "spatial-RNA.h5ad"
    spatac = sc.read_h5ad(atac_path)
    sprna = sc.read_h5ad(rna_path)

    # Convert weight matrices to DataFrames with peak/gene names as index
    W1_df = pd.DataFrame(
        W1,
        index=spatac.var.index,
        columns=[f'K{f+1}' for f in range(N_PROGRAMS)]
    )
    W2_df = pd.DataFrame(
        W2,
        index=sprna.var.index,
        columns=[f'K{f+1}' for f in range(N_PROGRAMS)]
    )

    # Restrict peak–gene distance table to peaks/genes present in current run
    distances = distances_orig[
        (distances_orig["peak_name"].isin(W1_df.index)) &
        (distances_orig["genes"].isin(W2_df.index))
    ]

    # ---------------------------
    # Construct positive pairs
    # ---------------------------
    # Define positives as peak–gene pairs within 10kb genomic distance
    pos_pairs = distances[distances["min_distance"] <= 10000][
        ["genes", "peak_name"]
    ].copy()

    # Set of all observed peak–gene pairs (used to avoid sampling true pairs as negatives)
    distance_set = set(zip(distances["peak_name"], distances["genes"]))

    # All possible peaks and genes in current model
    all_peaks = W1_df.index.values
    all_genes = W2_df.index.values

    n_pos = len(pos_pairs)  # Number of positive pairs

    # ---------------------------
    # Construct matched negative pairs
    # ---------------------------
    rng = np.random.default_rng(42)
    neg_list = []

    max_attempts = n_pos * 10  # Prevent infinite loop
    attempts = 0

    # Randomly sample peak–gene pairs not present in distance_set
    while len(neg_list) < n_pos and attempts < max_attempts:
        peak = rng.choice(all_peaks)
        gene = rng.choice(all_genes)

        # Ensure sampled pair is not a known genomic pair
        if (peak, gene) not in distance_set:
            neg_list.append((peak, gene))
        attempts += 1

    # Skip if no valid negative pairs were generated
    if len(neg_list) == 0:
        continue

    neg_pairs = pd.DataFrame(neg_list, columns=["peak_name", "genes"])

    # ---------------------------
    # Compute cosine similarity
    # ---------------------------
    # Cosine similarity between matched peak and gene program loadings
    cs_pos = cosine_similarity(
        W1_df.loc[pos_pairs["peak_name"]].values,
        W2_df.loc[pos_pairs["genes"]].values
    )

    cs_neg = cosine_similarity(
        W1_df.loc[neg_pairs["peak_name"]].values,
        W2_df.loc[neg_pairs["genes"]].values
    )

    # Contrast score = mean positive similarity − mean negative similarity
    contrast_score = np.diag(cs_pos).mean() - np.diag(cs_neg).mean()

    print(
        f"Rep {rep} | K {k} | lambda1 {lambda1} | "
        f"bs {batch_size} | Contrast: {contrast_score:.4f}"
    )

    # Store results
    results.append({
        "rep": rep,
        "K": k,
        "lambda1": lambda1,
        "batch_size": batch_size,
        "contrast_score": contrast_score
    })

Rep 45 | K 8 | lambda1 1 | bs 1024 | Contrast: 0.1080
Rep 45 | K 8 | lambda1 3 | bs 1024 | Contrast: 0.0876
Rep 45 | K 8 | lambda1 5 | bs 1024 | Contrast: 0.0386
Rep 45 | K 8 | lambda1 7 | bs 1024 | Contrast: 0.1072
Rep 45 | K 8 | lambda1 9 | bs 1024 | Contrast: 0.0578
Rep 45 | K 8 | lambda1 11 | bs 1024 | Contrast: 0.0210
Rep 45 | K 8 | lambda1 13 | bs 1024 | Contrast: 0.0227
Rep 45 | K 8 | lambda1 15 | bs 1024 | Contrast: 0.0209
Rep 45 | K 16 | lambda1 1 | bs 1024 | Contrast: 0.0871
Rep 45 | K 16 | lambda1 3 | bs 1024 | Contrast: 0.0691
Rep 45 | K 16 | lambda1 5 | bs 1024 | Contrast: 0.0615
Rep 45 | K 16 | lambda1 7 | bs 1024 | Contrast: 0.0486
Rep 45 | K 16 | lambda1 9 | bs 1024 | Contrast: 0.0389
Rep 45 | K 16 | lambda1 11 | bs 1024 | Contrast: 0.0300
Rep 45 | K 16 | lambda1 13 | bs 1024 | Contrast: 0.0255
Rep 45 | K 16 | lambda1 15 | bs 1024 | Contrast: 0.0243
Rep 45 | K 32 | lambda1 1 | bs 1024 | Contrast: 0.0819
Rep 45 | K 32 | lambda1 3 | bs 1024 | Contrast: 0.0759
Rep 45 | K 3

KeyboardInterrupt: 

In [13]:
# ---------------------------
# Hyperparameter settings
# ---------------------------

LAMBDA1_VALUES = [1, 3, 5, 7, 9, 11, 13, 15]  # Regularization strengths
REP = 45                                     # Simulation replicate ID
K = 32                                       # Number of latent programs
BATCH_SIZE = 1024                            # Training batch size

BASE_DIR = Path("./new_sim")                 # Base directory containing outputs
N_PROGRAMS = K                               # Number of programs to use

# Store contrast score results
results = []

# Iterate over lambda1 values only
for lambda1 in LAMBDA1_VALUES:

    # Construct directory path
    run_dir = (
        BASE_DIR
        / f"output_sim_p_rep_{REP}"
        / f"lambda1_{lambda1}_bs_{BATCH_SIZE}_k_{K}"
    )

    # Required file paths
    atac_path = run_dir / "spatial-ATAC.h5ad"
    weights_path = run_dir / "weights.pt"
    rna_path = run_dir / "spatial-RNA.h5ad"

    # Skip if files do not exist
    if not atac_path.exists() or not weights_path.exists():
        continue

    # Load trained model weights
    checkpoint = torch.load(weights_path, map_location="cpu")

    # Extract peak (W1) and gene (W2) program loadings
    W1 = checkpoint["W1"].numpy()[:, :N_PROGRAMS]
    W2 = checkpoint["W2"].numpy()[:, :N_PROGRAMS]

    # Load spatial data
    spatac = sc.read_h5ad(atac_path)
    sprna = sc.read_h5ad(rna_path)

    # Convert to DataFrames with peak/gene names
    W1_df = pd.DataFrame(
        W1,
        index=spatac.var.index,
        columns=[f"K{i+1}" for i in range(N_PROGRAMS)]
    )

    W2_df = pd.DataFrame(
        W2,
        index=sprna.var.index,
        columns=[f"K{i+1}" for i in range(N_PROGRAMS)]
    )

    # Restrict distance table to current peaks/genes
    distances = distances_orig[
        (distances_orig["peak_name"].isin(W1_df.index)) &
        (distances_orig["genes"].isin(W2_df.index))
    ]

    # ---------------------------
    # Positive pairs (≤10 kb)
    # ---------------------------
    pos_pairs = distances[
        distances["min_distance"] <= 10000
    ][["genes", "peak_name"]].copy()

    if len(pos_pairs) == 0:
        continue

    distance_set = set(zip(distances["peak_name"], distances["genes"]))

    all_peaks = W1_df.index.values
    all_genes = W2_df.index.values
    n_pos = len(pos_pairs)

    # ---------------------------
    # Negative sampling
    # ---------------------------
    rng = np.random.default_rng(42)
    neg_list = []
    max_attempts = n_pos * 10
    attempts = 0

    while len(neg_list) < n_pos and attempts < max_attempts:
        peak = rng.choice(all_peaks)
        gene = rng.choice(all_genes)

        if (peak, gene) not in distance_set:
            neg_list.append((peak, gene))

        attempts += 1

    if len(neg_list) == 0:
        continue

    neg_pairs = pd.DataFrame(neg_list, columns=["peak_name", "genes"])

    # ---------------------------
    # Cosine similarity
    # ---------------------------
    cs_pos = cosine_similarity(
        W1_df.loc[pos_pairs["peak_name"]].values,
        W2_df.loc[pos_pairs["genes"]].values
    )

    cs_neg = cosine_similarity(
        W1_df.loc[neg_pairs["peak_name"]].values,
        W2_df.loc[neg_pairs["genes"]].values
    )

    # Contrast score
    contrast_score = np.diag(cs_pos).mean() - np.diag(cs_neg).mean()

    print(
        f"Rep {REP} | K {K} | lambda1 {lambda1} | "
        f"bs {BATCH_SIZE} | Contrast: {contrast_score:.4f}"
    )

    results.append({
        "rep": REP,
        "K": K,
        "lambda1": lambda1,
        "batch_size": BATCH_SIZE,
        "contrast_score": contrast_score
    })

# Convert to DataFrame for analysis
results_df = pd.DataFrame(results)

Rep 45 | K 32 | lambda1 1 | bs 1024 | Contrast: 0.0819
Rep 45 | K 32 | lambda1 3 | bs 1024 | Contrast: 0.0759
Rep 45 | K 32 | lambda1 5 | bs 1024 | Contrast: 0.0695
Rep 45 | K 32 | lambda1 7 | bs 1024 | Contrast: 0.0641
Rep 45 | K 32 | lambda1 9 | bs 1024 | Contrast: 0.0621
Rep 45 | K 32 | lambda1 11 | bs 1024 | Contrast: 0.0588
Rep 45 | K 32 | lambda1 13 | bs 1024 | Contrast: 0.0550
Rep 45 | K 32 | lambda1 15 | bs 1024 | Contrast: 0.0493


In [5]:
hp_cos=pd.DataFrame(results)